In [31]:
from gensim.models import KeyedVectors
from scipy import spatial
import pandas as pd
import numpy as np
import re

In [4]:
model = KeyedVectors.load_word2vec_format('../data/duyet_word2vec_skill.bin', binary=True)
candidate_index_path = '../data/candidate_skills_vector_test.csv'

In [54]:
def avg_feature_vector(words, model, num_features):
        #function to average all words vectors in a given paragraph
        featureVec = np.zeros((num_features,), dtype="float32")
        nwords = 0

        #list containing names of words in the vocabulary
        index2word_set = set(model.index2word) # this is moved as input param for performance reasons
        for word in words:
            if word in index2word_set:
                nwords = nwords+1
                featureVec = np.add(featureVec, model[word])

        if(nwords>0):
            featureVec = np.divide(featureVec, nwords)
        return featureVec
    
def find_best_candidate_by_skills(skills, number_of_result = 10):
    top_results = []
    input_skill_vector = avg_feature_vector(skills.split(), model=model, num_features=300)
    
    def update_result(top_results, candidate_id, score, number_of_result):
        top_results.append({ 'candidate_id': candidate_id, 'score': score })
        top_result_sorted = sorted(top_results, key=lambda k: k['score'], reverse=True) 
        top_results = top_result_sorted[:number_of_result]
        return top_results

    chunksize = 10 ** 5
    for candidates in pd.read_csv(candidate_index_path, chunksize=chunksize, error_bad_lines=False):
        for i, candidate in candidates.iterrows():
            candidate_feature = candidate[2]
            candidate_feature = re.sub(' +', ' ', candidate_feature)
            candidate_feature = candidate_feature.replace('\n', '').replace('[', '').replace(']', '').strip().split(' ')
            candidate_feature = np.array(candidate_feature, dtype="float32")
            
            similarity =  1 - spatial.distance.cosine(input_skill_vector, candidate_feature)
            top_results = update_result(top_results, candidate_id=candidate[0], score=similarity, number_of_result=number_of_result)
    return top_results

In [55]:
find_best_candidate_by_skills("java c html")

[{'candidate_id': 1446, 'score': 0.90957513929566125},
 {'candidate_id': 24242, 'score': 0.90686816702549411},
 {'candidate_id': 22136, 'score': 0.90454623840576864},
 {'candidate_id': 22122, 'score': 0.89931653774642262},
 {'candidate_id': 28724, 'score': 0.89899006430681438},
 {'candidate_id': 11904, 'score': 0.8988085700936127},
 {'candidate_id': 29372, 'score': 0.89493568996101491},
 {'candidate_id': 2890, 'score': 0.89356782256672496},
 {'candidate_id': 24272, 'score': 0.88942238122786865},
 {'candidate_id': 4660, 'score': 0.88897123928621014}]

In [53]:
sorted.__getattribute__

<method-wrapper '__getattribute__' of builtin_function_or_method object at 0x7fef6534ad40>